In [57]:
## Import required and advanced modules
from selenium import webdriver
from IPython.core.display import clear_output
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd
from datetime import date

In [36]:
def scrape_reviews(link,provider,view_more_times):
    ## Initialize empty list of variables to scrape
    review = []
    course_title = []
    reviewed_by = []
    stars = []
    
    # Initialize driver
    driver = webdriver.Chrome()
    driver.get(link)
    driver.maximize_window()
    wait = WebDriverWait(driver, 25)
    time.sleep(4)

    # Click on reviews button
    elem = driver.find_element_by_xpath('//*[@id="reviews"]/div[1]/span')
    #elem.click()

    # Click "view more" page
    try:
        for i in range(1,view_more_times):
            time.sleep(4)
            elem = driver.find_element_by_xpath('//*[@id="providerCourseReviews"]/div[3]/a')
            elem.click()
    except Exception as e:
        print(e)

    # Extract title for which review has been provided
    course_title_tag = driver.find_elements(By.CSS_SELECTOR,'div.heading.mt-2.ml-1.mr-1')
    for tlt in course_title_tag:
        course_title.append(tlt.text if course_title_tag is not None else 'missing')
    
    # Get the review
    review_tag = driver.find_elements(By.CSS_SELECTOR,'div.mt-1.ml-1.mr-1 p')
    for rvw in review_tag:
        review.append(rvw.text if review_tag is not None else 'missing')
    
#     reviewed_by_tag = driver.find_elements(By.CSS_SELECTOR,'div.mt-1.text-muted.subtext.d-md-flex.ml-1.mr-1')
#     for rvby in reviewed_by_tag:
#         reviewed_by.append(rvby.text if reviewed_by_tag is not None else 'missing')
    
    # Get the no of stars
    stars_tag = driver.find_elements(By.CSS_SELECTOR,'article h3.sr-only')
    for strs in stars_tag:
        try:
            stars.append(strs.text)
        except:
            stars.append('missing')
    
    # Create df with scraped variables
    df = pd.DataFrame({
        "course":course_title,
        "review":review,
#         "reviewed_by":reviewed_by,
         "stars":stars
    })
    
    # Extract purchase time.
    #df['purchase_time'] = df.reviewed_by.str.split("|").str[1]
    # Insert provider
    df['provider'] = provider
    today = date.today().strftime('%d_%b')
    return df.to_csv(f"{provider}_review_today.csv",index=False)

In [18]:
# Scrape one education reviews
scrape_reviews('https://www.reed.co.uk/courses/profile/one-education','one_education',23)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="providerCourseReviews"]/div[3]/a"}
  (Session info: chrome=79.0.3945.130)



In [37]:
# Scrape janets reviews
scrape_reviews('https://www.reed.co.uk/courses/profile/janets','janets',10)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="providerCourseReviews"]/div[3]/a"}
  (Session info: chrome=79.0.3945.130)



In [38]:
# Scrape course gate reviews
scrape_reviews('https://www.reed.co.uk/courses/profile/course-gate','course_gate',12)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="providerCourseReviews"]/div[3]/a"}
  (Session info: chrome=79.0.3945.130)



In [50]:
## Read all the dfs and concate them for processing and cleaning
df_one = pd.read_csv('one_education_review_today.csv')
df_janets = pd.read_csv('janets_review_today.csv')
df_cg = pd.read_csv('course_gate_review_today.csv')
## Merge dfs
df = pd.concat([df_one,df_janets,df_cg],axis=0,sort=False).reset_index(drop=True)

In [53]:
# Get only stars in digit and overwrite the current column
df['stars'] = df.stars.str[0:3]

# Drop some columns
df = df.drop(['reviewed_by','purchase_time'],axis=1)
df.head()

,course,review,stars,provider
0,Sage 50 Accounting 2018 for Beginner - CPD Acc...,Enjoying the course. Well worth the cost. A qu...,4.0,one_education
1,HR and Payroll Management with Recruitment Con...,Good value for money course. Informative and g...,4.6,one_education
2,Building Surveyor Training - CPD Accredited,I have not been provided with any login detail...,1.0,one_education
3,Transport Planner Course - CPD Accredited,NaN,4.3,one_education
4,Chakra Healing and Balancing Online Training,"If I ordered this, it was a mistake. I certain...",1.0,one_education


In [56]:
df.to_csv('reed_review.csv',index=False)